In [ ]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 

In [ ]:
SOURCE_FOLDER_TRAIN= '../AudioData/audio-cats-and-dogs/cats_dogs/train/'
OUTPUT_FOLDER_TRAIN = '../GeneratedData/audio-cats-and-dogs/cats_dogs/train/'

In [ ]:
SOURCE_FOLDER_TEST= '../AudioData/audio-cats-and-dogs/cats_dogs/test/'
OUTPUT_FOLDER_TEST = '../GeneratedData/audio-cats-and-dogs/cats_dogs/test/'

In [ ]:
### Image output resolution
RESOLUTION = 224
### Spectograph resolution
SAMPLE_RATE = 22050
N_FFT = 1024
HOP_LENGTH = 256
N_MELS = RESOLUTION
FMIN = 20
FMAX = SAMPLE_RATE / 2 

In [ ]:
def log_mel_spec_tfm(src_path, dst_path):
    print(src_path)
    fname=src_path.split('/')[-1];
    x=[]
    sample_rate=0
    try:
        channels ,sample_rate_in = librosa.load(src_path,mono=False)
    except:
        print(fname+" Could not be computed!")
        return
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels))))
    channelcounter=0
    for x in channels:
        channelcounter+=1
        audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
        mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=N_FFT, 
                                                        hop_length=HOP_LENGTH, 
                                                        n_mels=N_MELS, power=2.0,
                                                       fmin=FMIN,fmax=FMAX)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
        width = np.shape(mel_spec_db)[1]
        if width >= n_mels:
            for i in range(0,int(np.floor(width/n_mels))):
                out= np.zeros(shape=(RESOLUTION,RESOLUTION))
                if((i+1)*n_mels<=width):
                    out = mel_spec_db[:,(i)*n_mels:(i+1)*n_mels]
                    im = plt.imshow(out)
                    colors = im.cmap(im.norm(out))
                    data = colors.astype(np.float64) / np.max(colors) # normalize the data to 0 - 1
                    data = 255 * data # Now scale by 255
                    img = data.astype(np.uint8)
                    out = img[:,:,0:3] 
                plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), out) 

In [ ]:
def ComputeSpectrograms(IN_FOLDER,OUT_FOLDER):
    SourceFoldersLabels = [f.path for f in os.scandir(IN_FOLDER) if f.is_dir()]
    for path in SourceFoldersLabels:
        FileList = [f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))]
        Label = path.split('/')[-1]
        outFolder = os.path.join(OUT_FOLDER,Label)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        with multiprocessing.Pool(4) as p:
            p.starmap(log_mel_spec_tfm, [(f,outFolder) for f in FileList])
        print("Finished this pool. Now next Folder!")
    print("Done generating spectra!")  

In [ ]:
ComputeSpectrograms(SOURCE_FOLDER_TRAIN,OUTPUT_FOLDER_TRAIN)
ComputeSpectrograms(SOURCE_FOLDER_TEST,OUTPUT_FOLDER_TEST)

In [ ]:
ComputeSpectrograms("../AudioData/urban-sound-classification/train/Train/","../GeneratedData/urban-sound-classification/train/Train/")

In [ ]:
'''
Unused function
'''
def generateOutputFolder(overwrite, path,labels_local):
    if not os.path.exists(path):
        #if(overwrite):
            #TODO actually delete the folder
        os.makedirs(path)
    TemporaryDict={}
    for i in labels_local:
        LabeldDirectory = os.path.join(path,i)
        if not os.path.exists(LabeldDirectory):
            os.makedirs(LabeldDirectory)
        TemporaryDict[i]=LabeldDirectory
    return TemporaryDict